## Browsing through DEA data collections

#### A "datacube" is a digital information architecture that specialises in hosting and cataloguing spatial information. [Digital Earth Australia (DEA)](https://www.ga.gov.au/dea) is based on the [Open Data Cube](https://www.opendatacube.org/) infrastructure, and specialises in storing remotely sensed data from Earth Observation satellites such as [Landsat](https://landsat.gsfc.nasa.gov/) and [Sentinel-2](https://www.copernicus.eu/en/about-copernicus/infrastructure/discover-our-satellites).

#### The DEA datacube contains both raw satellite data and derived "products". These data products are often composed of a range of "measurements" such as the suite of remote sensing band values or statistical product summaries. Before running a query to load data from the datacube, it is useful to know what it contains. This notebook demonstrates several straightforward ways to inspect the product and measurement contents of a datacube.

#### Let's get started by loading up the usual Python modules and some extra ones that will allow us to demonstrate new functionality in this notebook.

In [1]:
import datacube
from check_answer import check_answer
import pandas as pd
from odc.ui import DcViewer

# Set some configurations for displaying tables nicely
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', None)

# Create the dc Datacube object, our handler for working with DEA
dc = datacube.Datacube(app="DEA_Products_and_measurements")

#### We already seen in the previous tutorial how to list the DEA data collections using the `dc.list_products()` function. 

#### Products listed under **name** in the following table represent the product options available when querying the datacube. 
The table below provides some useful information about each product, including a brief product **description**, the **instrument** and **platform** the data originated from (e.g. Landsat 8 OLI), and the product's default **crs** (coordinate reference system) and **resolution** if applicable.

> Note: For a comprehensive product description and access to complete product metadata, users are directed to the Geoscience Australia [Content Management Interface](https://cmi.ga.gov.au)

In [2]:
products = dc.list_products()

display_columns = ["name",
                   "description",
                   "default_crs",
                   "default_resolution"]

products[display_columns].sort_index()

,name,description,default_crs,default_resolution
name,,,,
aster_aloh_group_composition,aster_aloh_group_composition,ASTER,None,None
aster_aloh_group_content,aster_aloh_group_content,ASTER,None,None
aster_false_colour,aster_false_colour,ASTER,None,None
aster_feoh_group_content,aster_feoh_group_content,ASTER,None,None
aster_ferric_oxide_composition,aster_ferric_oxide_composition,ASTER,None,None
aster_ferric_oxide_content,aster_ferric_oxide_content,ASTER,None,None
aster_ferrous_iron_content_in_mgoh,aster_ferrous_iron_content_in_mgoh,ASTER,None,None
aster_ferrous_iron_index,aster_ferrous_iron_index,ASTER,None,None
aster_green_vegetation,aster_green_vegetation,ASTER,None,None


#### The previous `dc.list_products()` function provides information about the DEA products together with some high level and storage details. Each product contains a series of measurements. These can be individual satellite reflectance bands (e.g. Landsat's near-infrared band), derived products or statistical product summaries.

#### The `dc.list_measurements()` function can be used to interrogate the measurements associated with a given product (specified by the **name** column from the table above). For example, `ga_ls5t_ard_3` refers to the Geoscience Australia Landsat 5 Analysis-ready data Collection 3 product.

#### The table below includes a range of technical information about each band in the `ga_ls5t_ard_3` dataset, including any **aliases** which can be used to load the data, the data type or **dtype**, any **flags_definition** that are associated with the measurement (this information is used for tasks like cloud masking), and the measurement's **nodata** value.

In [3]:
product = "ga_ls5t_ard_3"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition,spectral_definition
measurement,,,,,,,
nbart_blue,nbart_blue,int16,1,-999,"[nbart_band01, blue]",NaN,NaN
nbart_green,nbart_green,int16,1,-999,"[nbart_band02, green]",NaN,NaN
nbart_red,nbart_red,int16,1,-999,"[nbart_band03, red]",NaN,NaN
nbart_nir,nbart_nir,int16,1,-999,"[nbart_band04, nir]",NaN,NaN
nbart_swir_1,nbart_swir_1,int16,1,-999,"[nbart_band05, swir_1, swir1]",NaN,NaN
nbart_swir_2,nbart_swir_2,int16,1,-999,"[nbart_band07, swir_2, swir2]",NaN,NaN
oa_fmask,oa_fmask,uint8,1,0,[fmask],"{'fmask': {'bits': [0, 1, 2, 3, 4, 5, 6, 7], 'values': {'0': 'nodata', '1': 'valid', '2': 'cloud', '3': 'shadow', '4': 'snow', '5': 'water'}, 'description': 'Fmask'}}",NaN
oa_nbart_contiguity,oa_nbart_contiguity,uint8,1,255,[nbart_contiguity],"{'contiguous': {'bits': [0], 'values': {'0': False, '1': True}}}",NaN
oa_azimuthal_exiting,oa_azimuthal_exiting,float32,1,NaN,[azimuthal_exiting],NaN,NaN


#### Exercise 1.1: Can you use the `list_measurements()` function above to show information relative to the `wofs_albers` product? 

#### If you remember from the bit flags tutorial, this product encodes classes encoded as bits into a 16-bit integer. Can you determine how WOfS encodes dry conditions? 

* **a)** Bit 0 is set to 0
* **b)** Bit 0 is set to 1
* **c)** All bits in positions [7, 6, 5, 4, 3, 1, 0] are set to 1
* **d)** All bits in positions [7, 6, 5, 4, 3, 1, 0] are set to 0

In [9]:

product = "wofs_albers"
measurements = dc.list_measurements()
measurements.loc[product]
# Substitute the ? symbols by either 'a', 'b', 'c' or 'd'
answ = 'd'

check_answer("DEA1.1", answ)

'Well done!'

#### The interactive `DcViewer` utility provides a more visual way of exploring the data that is available within the Digital Earth Australia datacube. 

#### After running the cell below, select a product from the drop-down menu on the top-right of the map to show the areas where data are available in blue. Use the back and forward buttons above the map to toggle through time.

>Note: The utility is only able to visualise a limited number of datasets at one time.
If the available data footprints do not appear, either press the "show" button on the top right, or zoom in on the map.

In [14]:
DcViewer(dc=dc, 
         time='2015', 
         width='800px',
         center=(-27.48, 153.10),
         zoom=7)

#### Another way to view the available data within a datacube is to visit the DEA Datacube Explorer sites. These webpages visualise the data that is available for every product in DEA.

#### There is a different Datacube Explorer page for both the NCI and DEA Sandbox environments to account for the different datasets that are available in each of these datacubes:

* [NCI DEA Datacube Explorer](http://explorer.dea.ga.gov.au)
* [DEA Sandbox Datacube Explorer](http://explorer.sandbox.dea.ga.gov.au)

![DEA Explorer](data/DEAExplorer.JPG)

#### Exercise 1.2: Using the previous link to the DEA Sandbox Datacube Explorer, can you locate the collection named `ls8_fc_albers` and find the names of the measurements it contains on bottom right side?

#### Disclaimer: The original notebook for this tutorial has been taken from the Sandbox `Beginners_guide` folder. Refer to the conditions specified in the original notebook and for updated versions of the code.